# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

1. We've provided a csv file containing data about the game show _Jeopardy!_ in a file named `jeopardy.csv`. Load the data into a DataFrame and investigate its contents. Try to print out specific columns.

   Note that in order to make this project as "real-world" as possible, we haven't modified the data at all - we're giving it to you exactly how we found it. As a result, this data isn't as "clean" as the datasets you normally find on Codecademy. More specifically, there's something odd about the column names. After you figure out the problem with the column names, you may want to rename them to make your life easier for the rest of the project.
   
   In order to display the full contents of a column, we've added this line of code for you:
   
   ```py
   pd.set_option('display.max_colwidth', None)
   ```

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
questions = pd.read_csv('jeopardy.csv')
questions.columns = questions.columns.str.strip()

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [2]:
def filter_questions_by_words(df, words):
    """
    Filters the dataset for rows where the 'Question' column contains all the words in the given list.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the dataset.
        words (list): A list of words to filter the questions by.

    Returns:
        pd.DataFrame: A filtered DataFrame containing rows where all the words appear in the 'Question' column.
    """
    # Ensure the column name is consistent
    if 'Question' not in df.columns:
        raise KeyError("The dataset does not contain a 'Question' column.")
    
    # Apply the filter
    for word in words:
        df = df[df['Question'].str.contains(word, case=False, na=False)]
    return df

words_to_filter = ["King", "England"]
filtered_questions = filter_questions_by_words(questions, words_to_filter)

# Print the filtered questions
print(filtered_questions["Question"].head(5))

4953                   Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337     In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                   This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710              This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                      It's the number that followed the last king of England named William
Name: Question, dtype: object


3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [3]:
def filter_questions_by_words(df, words):
    """
    Filters the dataset for rows where the 'Question' column contains all the words in the given list.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the dataset.
        words (list): A list of words to filter the questions by.

    Returns:
        pd.DataFrame: A filtered DataFrame containing rows where all the words appear in the 'Question' column.
    """
    # Tokenize and clean each question to avoid substring matches
    def contains_all_words(question, words):
        # Split question into words, removing punctuation and lowercasing
        tokens = set(word.strip(".,'\"!?").lower() for word in question.split())
        # Check if all words are present in the tokens
        return all(word.lower() in tokens for word in words)

    # Apply the filtering logic
    return df[df['Question'].apply(lambda q: contains_all_words(q, words))]

words_to_filter = ["king", "EngLand"]
filtered_questions = filter_questions_by_words(questions, words_to_filter)

# Print the filtered questions
print(filtered_questions["Question"].head(5))

6337     In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                   This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710              This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                      It's the number that followed the last king of England named William
14912         This country's King Louis IV was nicknamed "Louis From Overseas" because he was raised in England
Name: Question, dtype: object


4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [4]:
def convert_to_float(value):
    if isinstance(value, str):  # Check if the value is a string
        try:
            # Remove dollar signs and commas, and convert to float
            return float(value.replace('$', '').replace(',', ''))
        except ValueError:
            return None
    return value  # Return the value as-is if it's already a float or None

# Apply the function to "Value" column
questions["Value"] = questions["Value"].apply(convert_to_float)

print(questions[["Value"]].head(5))


   Value
0  200.0
1  200.0
2  200.0
3  200.0
4  200.0


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [5]:
def count_unique_answers(df):
    """
    Counts the occurrences of unique answers in the dataset.

    Parameters:
        df (pd.DataFrame): The filtered DataFrame containing a column named 'Answer'.

    Returns:
        pd.Series: A series with unique answers as the index and their counts as values.
    """
    if 'Answer' not in df.columns:
        raise KeyError("The dataset must contain an 'Answer' column.")

    # Count occurrences of each unique answer
    return df['Answer'].value_counts()

# Filter the dataset for questions containing the word "King"
words_to_be_filtered = filter_questions_by_words(questions, ["King"])

# Get the count of unique answers
unique_answer_counts = count_unique_answers(words_to_be_filtered)

# Display the most common answers
print(unique_answer_counts.head(5))  # Show the top 10 most common answers



Answer
Henry VIII    44
Solomon       27
Louis XIV     25
Sweden        24
David         24
Name: count, dtype: int64


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [6]:
def filter_questions_by_date_and_word(df, start_year, end_year, word):
    """
    Filters questions in a given date range that contain a specific word.

    Parameters:
        df (pd.DataFrame): The dataset.
        start_year (int): Start year of the date range.
        end_year (int): End year of the date range.
        word (str): Word to filter the questions by.

    Returns:
        pd.DataFrame: A filtered DataFrame containing questions that match the criteria.
    """
    # Convert 'Air Date' to datetime format if not already
    df['Air Date'] = pd.to_datetime(df['Air Date'])

    # Filter by date range and word
    filtered_df = df[
        (df['Air Date'].dt.year >= start_year) &
        (df['Air Date'].dt.year <= end_year) &
        (df['Question'].str.contains(word, case=False, na=False))
    ]
    return filtered_df

# Questions containing "Computer" in the 1990s
computer_90s = filter_questions_by_date_and_word(questions, 1990, 1999, "Computer")

# Questions containing "Computer" in the 2000s
computer_2000s = filter_questions_by_date_and_word(questions, 2000, 2009, "Computer")

# Print the counts
print(f"Questions in the 1990s with 'Computer': {len(computer_90s)}")
print(f"Questions in the 2000s with 'Computer': {len(computer_2000s)}")


Questions in the 1990s with 'Computer': 98
Questions in the 2000s with 'Computer': 268


In [7]:
def category_by_round_analysis(df):
    """
    Analyzes the connection between the round and the category.

    Parameters:
        df (pd.DataFrame): The dataset.

    Returns:
        pd.DataFrame: A table showing the frequency of categories per round.
    """
    # Create a pivot table of rounds vs categories
    pivot_table = df.pivot_table(
        index='Category', 
        columns='Round', 
        aggfunc='size', 
        fill_value=0
    )
    return pivot_table

# Analyze the connection
category_round_table = category_by_round_analysis(questions)

# Inspect specific categories
print(category_round_table.loc['LITERATURE'])


Round
Double Jeopardy!    381
Final Jeopardy!      10
Jeopardy!           105
Tiebreaker            0
Name: LITERATURE, dtype: int64
